In [2]:
! pip install transformers
! pip install datasets

In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("siebert/sentiment-roberta-large-english")
model = AutoModelForSequenceClassification.from_pretrained("siebert/sentiment-roberta-large-english")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

In [4]:
from collections import defaultdict, Counter
import json

import numpy as np
import matplotlib.pyplot as plt
import torch

In [5]:
def print_encoding(model_inputs, indent =4):
  indent_str = " "*indent
  print("{")

  for k,v in model_inputs.items():
    print(indent_str + k + ":")
    print(indent_str + indent_str + str(v))
    print("}")

In [6]:
inputs = "I'm excited to learn about Transformers!"
tokenized_inputs = tokenizer(inputs,return_tensors="pt")  # pt means pytorch tensors
outputs = model(**tokenized_inputs)

In [7]:
labels = ['NEGATIVE', 'POSITIVE']
prediction = torch.argmax(outputs.logits)

In [8]:
torch.softmax(outputs.logits, dim =1)

tensor([[0.0012, 0.9988]], grad_fn=<SoftmaxBackward0>)

In [9]:
print(f"Input: {inputs}\n")
print(f"Tokenized Inputs: {tokenized_inputs}\n")
print(f"Model Outputs:")
print(outputs)
print(f"The Prediction is: {labels[prediction]}")

Input: I'm excited to learn about Transformers!

Tokenized Inputs: {'input_ids': tensor([[    0,   100,   437,  2283,     7,  1532,    59, 34379,   328,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

Model Outputs:
SequenceClassifierOutput(loss=None, logits=tensor([[-3.7800,  2.9480]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
The Prediction is: POSITIVE


---
---

## Tokenizers

In [10]:
from transformers import DistilBertTokenizer,DistilBertTokenizerFast,AutoTokenizer
name = "distilbert/distilbert-base-cased"
tokenizer = DistilBertTokenizer.from_pretrained(name)
print(tokenizer)

tokenizer = AutoTokenizer.from_pretrained(name)
print(tokenizer)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

DistilBertTokenizer(name_or_path='distilbert/distilbert-base-cased', vocab_size=28996, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)
DistilBertTokenizerFast(name_or_path='distilbert/distilbert-base-cased', vocab_size=28996, model_max_length=5

In [11]:
# call the tokenizer
input_str = "Hugging Face is Great!"
tokenized_inputs = tokenizer(input_str, return_tensors ='pt')

In [12]:
print_encoding(tokenized_inputs)

{
    input_ids:
        tensor([[  101, 20164, 10932, 10289,  1110,  2038,   106,   102]])
}
    attention_mask:
        tensor([[1, 1, 1, 1, 1, 1, 1, 1]])
}


In [13]:
print(tokenized_inputs.input_ids)
print(tokenized_inputs["input_ids"])

tensor([[  101, 20164, 10932, 10289,  1110,  2038,   106,   102]])
tensor([[  101, 20164, 10932, 10289,  1110,  2038,   106,   102]])


In [14]:
cls = [tokenizer.cls_token_id]
sep = [tokenizer.sep_token_id]

# Tokenization Steps:
input_tokens = tokenizer.tokenize(input_str)
input_ids = tokenizer.convert_tokens_to_ids(input_tokens)
input_ids_special_tokens = cls + input_ids + sep

decoded_str = tokenizer.decode(input_ids_special_tokens)

print("start:                 ", input_str)
print("tokenize:              ", input_tokens)
print("convert_tokens_to_ids: ", input_ids)
print("add special tokens:    ", input_ids_special_tokens)
print("decode:                ", decoded_str)

start:                  Hugging Face is Great!
tokenize:               ['Hu', '##gging', 'Face', 'is', 'Great', '!']
convert_tokens_to_ids:  [20164, 10932, 10289, 1110, 2038, 106]
add special tokens:     [101, 20164, 10932, 10289, 1110, 2038, 106, 102]
decode:                 [CLS] Hugging Face is Great! [SEP]


In [15]:
tokenizer.decode([10289])

'Face'

In [16]:
tokenizer.decode([2038])

'Great'

In [17]:
tokenizer.decode([0])  # padding

'[PAD]'

In [18]:
tokenizer.decode([100])   # unknown

'[UNK]'

In [19]:
# The toknizers can return pytorch tensors
model_inputs = tokenizer("NPTEL is great", return_tensors='pt')
print("PyTorch Tensors:")
print_encoding(model_inputs)


PyTorch Tensors:
{
    input_ids:
        tensor([[  101,   151,  2101, 12880,  2162,  1110,  1632,   102]])
}
    attention_mask:
        tensor([[1, 1, 1, 1, 1, 1, 1, 1]])
}


In [20]:
model_inputs = tokenizer(["NPTEL is great",
                           "Learning Transformers is a fun",
                           " I will learn about RHLF next week"],
                         return_tensors = 'pt',
                         padding = True,
                         truncation = True)
print(f"Pad Tokens: {tokenizer.pad_token} | Pad Token ID: {tokenizer.pad_token_id}")
print("Padding")
print_encoding(model_inputs)


Pad Tokens: [PAD] | Pad Token ID: 0
Padding
{
    input_ids:
        tensor([[  101,   151,  2101, 12880,  2162,  1110,  1632,   102,     0,     0,
             0],
        [  101,  9681, 25267,  1110,   170,  4106,   102,     0,     0,     0,
             0],
        [  101,   146,  1209,  3858,  1164,   155,  9530,  2271,  1397,  1989,
           102]])
}
    attention_mask:
        tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
}


In [21]:
# You can also decode a whole batch at once
print("Batch Decode:")
print(tokenizer.batch_decode(model_inputs.input_ids))
print()
print("Batch Decode:(no special charecters)")
print(tokenizer.batch_decode(model_inputs.input_ids, skip_special_tokens = True))

Batch Decode:
['[CLS] NPTEL is great [SEP] [PAD] [PAD] [PAD]', '[CLS] Learning Transformers is a fun [SEP] [PAD] [PAD] [PAD] [PAD]', '[CLS] I will learn about RHLF next week [SEP]']

Batch Decode:(no special charecters)
['NPTEL is great', 'Learning Transformers is a fun', 'I will learn about RHLF next week']


In [27]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [29]:
!jupyter nbconvert --clear-output --inplace "/content/drive/MyDrive/Colab Notebooks/Hugging Face/Sentiment_Roberta.ipynb"

[NbConvertApp] Converting notebook /content/drive/MyDrive/Colab Notebooks/Hugging Face/Sentiment_Roberta.ipynb to notebook
[NbConvertApp] Writing 135309 bytes to /content/drive/MyDrive/Colab Notebooks/Hugging Face/Sentiment_Roberta.ipynb
